In [1]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram, butter, filtfilt
from concurrent.futures import ThreadPoolExecutor

# Extracting all channels data at once
def get_all_channels_data(abf):
    abf.setSweep(sweepNumber=0)
    data =  np.array([abf.sweepY for i in range(abf.channelCount)])
    time = np.array(abf.sweepX)
    return time, data 

# Function to plot raw signals
def plot_raw_signals(time,data):
    fig, axes = plt.subplots(len(data), 1, figsize=(20, 20), sharex=True)
    for i, ax in enumerate(axes):
        ax.plot(time, data[i])
        ax.set_title(f'Channel {i + 1}')
        ax.set_ylabel('Amplitude')
    axes[-1].set_xlabel('Time (s)')
    plt.show()

def compute_spectrogram(data, fs):
    f, t, Sxx = spectrogram(data + 1e-10, fs=fs)
    Sxx[Sxx == 0] = 1e-10  # Replace zero values with a small constant
    return f, t, 10 * np.log10(Sxx)

def plot_spectrograms(time, data):
    fig, axes = plt.subplots(len(data), 1, figsize=(20, 15), sharex=True)
    fs = 1/(time[1] - time[0])
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(compute_spectrogram, data, [fs]*len(data)))
    for i, (f, t, Sxx) in enumerate(results):
        ax = axes[i]
        cax = ax.pcolormesh(t, f, Sxx, shading='gouraud')
        ax.set_title(f'Spectrogram - Channel {i + 1}')
        ax.set_ylabel('Frequency (Hz)')
        fig.colorbar(cax, ax=ax, label='Power/Frequency (dB/Hz)')
    axes[-1].set_xlabel('Time (s)')
    plt.show()

# Function to apply a bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Function to plot filtered signals
def plot_filtered_signals(time, data, lowcut, highcut, band_name):
    fs = 1/(time[1] - time[0])
    filtered_data = np.array([bandpass_filter(channel_data, lowcut, highcut, fs) for channel_data in data])
    fig, axes = plt.subplots(len(data), 1, figsize=(10, 20), sharex=True)
    for i, ax in enumerate(axes):
        ax.plot(time, filtered_data[i])
        ax.set_title(f'{band_name} Band ({lowcut}-{highcut} Hz) - Channel {i + 1}')
        ax.set_ylabel('Amplitude')
    axes[-1].set_xlabel('Time (s)')
    plt.show()

In [2]:
# Load the ABF file
file_path = "2022_05_18_0001.abf"
abf = pyabf.ABF(file_path)
eeg_times, eeg_mv_data = get_all_channels_data(abf)
print(eeg_mv_data)
print(eeg_times)

AttributeError: 'ABF' object has no attribute 'close'

In [ ]:
# Plot raw signals
print(len(eeg_mv_data))
plot_raw_signals(eeg_times,eeg_mv_data)

In [ ]:
# Plot spectrograms
plot_spectrograms(eeg_times, eeg_mv_data)

In [ ]:
# Plot filtered signals for different frequency bands
# Example frequency bands: Delta (0.5-4 Hz), Theta (4-8 Hz), Alpha (8-12 Hz), Beta (12-30 Hz), Gamma (30-100 Hz)
bands = {'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 100)}
for band_name, (lowcut, highcut) in bands.items():
    plot_filtered_signals(eeg_times,eeg_mv_data, lowcut, highcut, band_name)